In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import datetime
from os import listdir
import tensorflow as tf
import random
#from get_dataset import read_npy_dataset, split_npy_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from get_models import unet_dense, get_segment_model, get_Discriminator, get_GAN, get_Generator, save_model

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
epochs = 5000
batch_size = 32
test_size = 0.1

In [4]:
# Dice Coefficient Loss Function
def dice_coefficient(y_true, y_pred):
    smoothing_factor = 1
    flat_y_true = y_true.flatten()
    flat_y_pred = y_pred.flatten()
    return (2. * np.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (np.sum(flat_y_true) + np.sum(flat_y_pred) + smoothing_factor)

In [5]:
def train_seg_model(model, splitted_npy_dataset_path, test_path, epochs, save_path):
   # test_XY = np.load(test_path+'/test.npy')
   # X_test, Y_test = test_XY[:,:,:,:,0:2],test_XY[:,:,:,:,2:3]
    
    
    batch_dirs = listdir(splitted_npy_dataset_path)
    len_batch_dirs = len(batch_dirs)

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    best_weights_path = save_path+'best_weights.h5'
    checkpoints = []
    checkpoints.append(EarlyStopping(monitor='val_loss', patience=500))
    checkpoints.append(ModelCheckpoint(best_weights_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False))
  #  checkpoints.append(TensorBoard(log_dir='Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))
    model.fit_generator(data_gen(splitted_npy_dataset_path, batch_size), steps_per_epoch=int(len_batch_dirs/batch_size)+1, epochs=5000, validation_data=data_gen(test_path, 16), 
                          validation_steps=5, callbacks=checkpoints)
    #scores = model.evaluate(X_test, Y_test)        
    #print(Y_predict.shape)
    #dice_score = dice_coefficient(model.predict(X_test), Y_test)
        
  #  print('Test loss:', scores[0], '\nTest accuracy:', scores[1], '\nDice Coefficient Accuracy:', dice_score)
    return model

In [6]:
def train_gan(Generator, Encoder, Discriminator, GAN, splitted_npy_dataset_path, test_path, epochs):
    # Dataset sample sharing:。
    start_time = datetime.datetime.now()
    batch_dirs = listdir(splitted_npy_dataset_path)
    batch_num = int(len(batch_dirs)/batch_size)

    X_test = np.load(test_path+'/structures.npy')
    Y_test = np.load(test_path+'/dose.npy')

    data_generator = data_gen(splitted_npy_dataset_path, batch_size)
    fake = np.ones((batch_size,))
    valid = np.zeros((batch_size,))
    
    for epoch in range(epochs):
        print('Epoch: {0}/{1}'.format(epoch+1, epochs))

        print('Discriminator Training:')
        # Discriminator Training:
        for i in range(int(batch_num)):
            # Getting generated data:
            
            X_batch, Y_batch = next(data_generator)
            fake_dose = Generator.predict(X_batch)

            # Getting real data:
          #  getted_real_data = next(data_generator)

            # Getting batch:
            # Xs:
         #   dis_batch_X1 = np.concatenate((X_batch, getted_real_data[0]), axis=-1)
         #   dis_batch_X2 = np.concatenate((generator_pred, getted_real_data[1]), axis=0)
            # Ys:
         #   dis_batch_Y = np.concatenate((np.zeros((getted_generator_data[0].shape[0], 1)), np.ones((getted_real_data[0].shape[0], 1))), axis=0)

            Discriminator.trainable = True
            d_loss_real = Discriminator.train_on_batch([X_batch,Y_batch],valid) 
            d_loss_fake = Discriminator.train_on_batch([X_batch,fake_dose],fake)
            d_loss = 0.5*np.add(d_loss_real,d_loss_fake)
           # Discriminator.fit([dis_batch_X1, dis_batch_X2], dis_batch_Y, batch_size=1, epochs=1, shuffle=True)

     #   print('Generator Training:')
        # Generator Training:
     #   for i in range(int(batch_num/2)):
            # Getting generated data:
           # getted_generator_data = next(data_generator)

            # Getting batch:
            # Xs:
           # gan_batch_X = getted_generator_data[0]
            # Ys:
           # gan_batch_Y = np.ones((gan_batch_X.shape[0], 1))
      #  for i in range(int(batch_num*2)):
            Discriminator.trainable = False
        #    Encoder.trainable = True
           # GAN.fit(gan_batch_X, gan_batch_Y, batch_size=1, epochs=1, shuffle=True)
            g_loss = GAN.train_on_batch([X_batch], [valid, Y_batch])
            elapsed_time = datetime.datetime.now() - start_time
            
        scores = Generator.evaluate(X_test, Y_test)
        print('\nTest Loss:', scores[0])
        save_model(Generator, path='Data_v3/Checkpoints/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v3/Checkpoints/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')

        print('Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"')

    return Generator, Encoder, Discriminator

In [7]:
def data_gen(splitted_npy_dataset_path, batch_size):
    c = 0
    batch_dirs = listdir(splitted_npy_dataset_path)
    n = os.listdir(splitted_npy_dataset_path) #List of training images
    random.shuffle(n)
    while True:
        X_batch = np.zeros((batch_size, 128, 256, 13)).astype('float')
        Y_batch = np.zeros((batch_size, 128, 256, 1)).astype('float')
        for i in range(c,c+batch_size):
            
            batch_XY = np.load(splitted_npy_dataset_path+'/'+n[i])
            #print('batch_XY size', batch_XY.shape)
            X_batch[i-c] =  batch_XY[:,:,:,0:13]
            Y_batch[i-c] =  batch_XY[:,:,:,13:14]    
        c = c + batch_size
        if(c+batch_size>=len(os.listdir(splitted_npy_dataset_path))):
            c=0;
            random.shuffle(n)
        yield X_batch, Y_batch

In [8]:
def main(train_gan_model = True):
    if train_gan_model:
        Generator = unet_dense(input_size = (128, 256, 13))
        Encoder = unet_dense(input_size = (128, 256, 13))
        Discriminator = get_Discriminator(input_shape_1=(128, 256, 13), input_shape_2=(128, 256, 1)) #, Encoder = Encoder)
        GAN = get_GAN((128, 256, 13), Generator, Discriminator)

        # Saving non-trained models:
        save_model(Generator, path='Data_v4/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Encoder, path='Data_v4/GAN-Models/Encoder/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v4/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')
        print('Non-Trained model saved to "Data/GAN-Models"!')
        ## Train
        Generator, Encoder, Discriminator = train_gan(Generator, Encoder, Discriminator, GAN, splitted_npy_dataset_path='Data_v4/npy_dataset/training_npy_dataset', test_path = 'Data_v4/npy_dataset/validation_npy', epochs = epochs)

        # Saving trained models:
        save_model(Generator, path='Data_v4/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Encoder, path='Data_v4/GAN-Models/Encoder/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v4/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')
        print('Trained model saved to "Data/GAN-Models"!')
        return Generator
        
    else:
        save_path = 'Data_v4/Checkpoints_UnetDense/'
        segment_model = unet_dense(input_size = (128, 256, 13))
        save_model(segment_model, path='Data_v4/Model_UnetDense/step2/', model_name = 'step2_model', weights_name = 'weights')
        print('Non-Trained model saved to "Data_v4/Model_UnetDense"!')
        model = train_seg_model(segment_model, splitted_npy_dataset_path='Data_v4/npy_dataset/training_step2', test_path = 'Data_v4/npy_dataset/validation_step2', epochs = epochs, save_path = save_path)
        save_model(model, path='Data_v4/Model_UnetDense/step2/', model_name = 'step2_model', weights_name = 'weights')
        print('Trained model saved to "Data_V2/Model_UnetDense"!')
        return segment_model

In [9]:
if __name__ == '__main__':
    main(train_gan_model = False)

output shape= (None, 128, 256, 1)
Dose prediction Model Architecture:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 256, 13 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 256, 16) 1888        input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 128, 256, 29) 0           input_1[0][0]                    
                                                                 conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)       

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 11 steps, validate for 5 steps
Epoch 1/5000
10/11 [==========================>...] - ETA: 1s - loss: 52.2358 - mean_squared_error: 52.2358 - acc: 0.0039
Epoch 00001: val_loss improved from inf to 42.75358, saving model to Data_v4/Checkpoints_UnetDense/best_weights.h5
11/11 [==============================] - 14s 1s/step - loss: 51.0775 - mean_squared_error: 51.0775 - acc: 0.0039 - val_loss: 42.7536 - val_mean_squared_error: 42.7536 - val_acc: 0.0039
Epoch 2/5000
10/11 [==========================>...] - ETA: 0s - loss: 30.2991 - mean_squared_error: 30.2991 - acc: 0.0038
Epoch 00002: val_loss improved from 42.75358 to 33.30562, saving model to Data_v4/Checkpoints_UnetDense/best_weights.h5
11/11 [==============================] - 6s 534ms/step - loss: 29.8410 - mean_squared_error: 29.8410 - acc: 0.0038 - val_loss: 33.3056 - val_mean_squared_error: 33.3056 - val_acc: 0.0039
Epoch 3/5000
10/11 [==========================>...] - ETA:

Epoch 21/5000
10/11 [==========================>...] - ETA: 0s - loss: 4.7524 - mean_squared_error: 4.7524 - acc: 0.0030
Epoch 00021: val_loss improved from 13.16568 to 12.61232, saving model to Data_v4/Checkpoints_UnetDense/best_weights.h5
11/11 [==============================] - 6s 502ms/step - loss: 4.7680 - mean_squared_error: 4.7680 - acc: 0.0030 - val_loss: 12.6123 - val_mean_squared_error: 12.6123 - val_acc: 0.0039
Epoch 22/5000
10/11 [==========================>...] - ETA: 0s - loss: 4.6233 - mean_squared_error: 4.6233 - acc: 0.0029
Epoch 00022: val_loss did not improve from 12.61232
11/11 [==============================] - 5s 493ms/step - loss: 4.5869 - mean_squared_error: 4.5869 - acc: 0.0030 - val_loss: 12.7031 - val_mean_squared_error: 12.7031 - val_acc: 0.0039
Epoch 23/5000
10/11 [==========================>...] - ETA: 0s - loss: 4.5129 - mean_squared_error: 4.5129 - acc: 0.0028
Epoch 00023: val_loss improved from 12.61232 to 12.42324, saving model to Data_v4/Checkpoints_U

Epoch 64/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.9610 - mean_squared_error: 2.9610 - acc: 7.4959e-05
Epoch 00064: val_loss did not improve from 8.65848
11/11 [==============================] - 5s 486ms/step - loss: 2.9572 - mean_squared_error: 2.9572 - acc: 7.5427e-05 - val_loss: 9.3938 - val_mean_squared_error: 9.3938 - val_acc: 6.1417e-05
Epoch 65/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.9817 - mean_squared_error: 2.9817 - acc: 7.6008e-05
Epoch 00065: val_loss did not improve from 8.65848
11/11 [==============================] - 5s 480ms/step - loss: 2.9707 - mean_squared_error: 2.9707 - acc: 7.8635e-05 - val_loss: 10.1862 - val_mean_squared_error: 10.1862 - val_acc: 9.4604e-05
Epoch 66/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.9431 - mean_squared_error: 2.9431 - acc: 8.3256e-05
Epoch 00066: val_loss did not improve from 8.65848
11/11 [==============================] - 5s 481ms/step - loss: 2.9503 - mean_squared_err

Epoch 86/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.6779 - mean_squared_error: 2.6779 - acc: 0.0022
Epoch 00086: val_loss improved from 8.04093 to 7.48215, saving model to Data_v4/Checkpoints_UnetDense/best_weights.h5
11/11 [==============================] - 5s 492ms/step - loss: 2.6652 - mean_squared_error: 2.6652 - acc: 0.0022 - val_loss: 7.4822 - val_mean_squared_error: 7.4822 - val_acc: 0.0018
Epoch 87/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.6736 - mean_squared_error: 2.6736 - acc: 0.0025
Epoch 00087: val_loss did not improve from 7.48215
11/11 [==============================] - 5s 491ms/step - loss: 2.6802 - mean_squared_error: 2.6802 - acc: 0.0025 - val_loss: 7.7523 - val_mean_squared_error: 7.7523 - val_acc: 0.0031
Epoch 88/5000
10/11 [==========================>...] - ETA: 0s - loss: 2.6885 - mean_squared_error: 2.6885 - acc: 0.0026
Epoch 00088: val_loss did not improve from 7.48215
11/11 [==============================] - 5s 480ms/s

...

Epoch 2844/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3058 - mean_squared_error: 0.3058 - acc: 0.0039
Epoch 02844: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 482ms/step - loss: 0.3064 - mean_squared_error: 0.3064 - acc: 0.0039 - val_loss: 2.9759 - val_mean_squared_error: 2.9759 - val_acc: 0.0039
Epoch 2845/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3150 - mean_squared_error: 0.3150 - acc: 0.0039
Epoch 02845: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 493ms/step - loss: 0.3138 - mean_squared_error: 0.3138 - acc: 0.0039 - val_loss: 2.8649 - val_mean_squared_error: 2.8649 - val_acc: 0.0039
Epoch 2846/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3144 - mean_squared_error: 0.3144 - acc: 0.0039
Epoch 02846: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 481ms/step - loss: 0.3151 - mean_squared_error: 0.3151 - acc: 0.0039

Epoch 2867/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3150 - mean_squared_error: 0.3150 - acc: 0.0039
Epoch 02867: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 488ms/step - loss: 0.3132 - mean_squared_error: 0.3132 - acc: 0.0039 - val_loss: 2.9201 - val_mean_squared_error: 2.9201 - val_acc: 0.0039
Epoch 2868/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3187 - mean_squared_error: 0.3187 - acc: 0.0039
Epoch 02868: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 494ms/step - loss: 0.3173 - mean_squared_error: 0.3173 - acc: 0.0039 - val_loss: 2.9890 - val_mean_squared_error: 2.9890 - val_acc: 0.0039
Epoch 2869/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3113 - mean_squared_error: 0.3113 - acc: 0.0039
Epoch 02869: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 482ms/step - loss: 0.3104 - mean_squared_error: 0.3104 - acc: 0.0039

Epoch 2890/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3194 - mean_squared_error: 0.3194 - acc: 0.0039
Epoch 02890: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 481ms/step - loss: 0.3204 - mean_squared_error: 0.3204 - acc: 0.0039 - val_loss: 3.1220 - val_mean_squared_error: 3.1220 - val_acc: 0.0039
Epoch 2891/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3093 - mean_squared_error: 0.3093 - acc: 0.0039
Epoch 02891: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 480ms/step - loss: 0.3093 - mean_squared_error: 0.3093 - acc: 0.0039 - val_loss: 3.0436 - val_mean_squared_error: 3.0436 - val_acc: 0.0039
Epoch 2892/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3142 - mean_squared_error: 0.3142 - acc: 0.0039
Epoch 02892: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 487ms/step - loss: 0.3145 - mean_squared_error: 0.3145 - acc: 0.0039

Epoch 2913/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3082 - mean_squared_error: 0.3082 - acc: 0.0039
Epoch 02913: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 481ms/step - loss: 0.3089 - mean_squared_error: 0.3089 - acc: 0.0039 - val_loss: 3.0927 - val_mean_squared_error: 3.0927 - val_acc: 0.0039
Epoch 2914/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3076 - mean_squared_error: 0.3076 - acc: 0.0039
Epoch 02914: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 490ms/step - loss: 0.3092 - mean_squared_error: 0.3092 - acc: 0.0039 - val_loss: 3.0296 - val_mean_squared_error: 3.0296 - val_acc: 0.0039
Epoch 2915/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3204 - mean_squared_error: 0.3204 - acc: 0.0039
Epoch 02915: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 474ms/step - loss: 0.3198 - mean_squared_error: 0.3198 - acc: 0.0039

Epoch 2936/5000
10/11 [==========================>...] - ETA: 0s - loss: 0.3047 - mean_squared_error: 0.3047 - acc: 0.0039
Epoch 02936: val_loss did not improve from 2.73036
11/11 [==============================] - 5s 480ms/step - loss: 0.3051 - mean_squared_error: 0.3051 - acc: 0.0039 - val_loss: 3.0188 - val_mean_squared_error: 3.0188 - val_acc: 0.0039
Trained model saved to "Data_V2/Model_UnetDense"!


In [19]:
# Training Segment Model:
def train_prediction_model(model, splitted_npy_dataset_path, test_path, epochs):
    
    batch_dirs = listdir(splitted_npy_dataset_path)
    len_batch_dirs = len(batch_dirs)

    if not os.path.exists('Data/Checkpoints/'):
        os.makedirs('Data/Checkpoints/')
    checkpoints = []
    checkpoints.append(ModelCheckpoint('Data/Checkpoints/best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True))
  #  checkpoints.append(TensorBoard(log_dir='Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))
    model.fit_generator(data_gen(splitted_npy_dataset_path, batch_size), steps_per_epoch=int(len_batch_dirs/batch_size)+1, epochs=100, validation_data=data_gen(test_path, 5), 
                          validation_steps=1, callbacks=checkpoints)
    scores = model.evaluate(X_test, Y_test)        
    print(Y_predict.shape)
    dice_score = dice_coefficient(model.predict(X_test), Y_test)
        
  #  print('Test loss:', scores[0], '\nTest accuracy:', scores[1], '\nDice Coefficient Accuracy:', dice_score)
    return model

In [ ]:
test = 